In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# code to append data to csv
def save_entry(filename,extra_df):
    try:
        df = pd.read_csv(filename)
    except:
        df = pd.DataFrame()
    df = pd.concat([df,extra_df])

    df.to_csv(filename,index=False)

In [3]:
# define the model to use, only one is used at a time, others must be commented.
# classifier name defines a column in the output.


# simple CNN classifier
"""def MNIST_classifier(ishape = (28,28,1),k = 10, lr = 1e-4):
    model_input = tf.keras.layers.Input(shape = ishape)
    l1 = tf.keras.layers.Conv2D(14,3,padding="same",activation="relu")(model_input)
    l2 = tf.keras.layers.MaxPool2D(2)(l1)
    l3 = tf.keras.layers.Conv2D(28,3,padding="same",activation="relu")(l2)
    flatten = tf.keras.layers.Flatten()(l3)
    out = tf.keras.layers.Dense(k,activation = 'softmax')(flatten)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy', metrics = ["accuracy"])
    return model 
classifier_name="simple_cnn"    
"""


# more complex CNN classifier
"""def MNIST_classifier(ishape = (28,28,1),k = 10, lr = 1e-4):
    model_input = tf.keras.layers.Input(shape = ishape)
    l1 = tf.keras.layers.Conv2D(48,3,padding="same",activation="relu")(model_input)
    l2 = tf.keras.layers.Conv2D(48,3,padding="same",activation="relu")(l1)
    l2_drop = tf.keras.layers.Dropout(0.25)(l2)
    l3 = tf.keras.layers.MaxPool2D(2)(l2_drop)
    l4 = tf.keras.layers.Conv2D(96,3,padding="same",activation="relu")(l3)
    l5 = tf.keras.layers.Conv2D(96,3,padding="same",activation="relu")(l4)
    l5_drop = tf.keras.layers.Dropout(0.25)(l5)
    flatten = tf.keras.layers.Flatten()(l5_drop)
    out = tf.keras.layers.Dense(k,activation = 'softmax')(flatten)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy', metrics = ["accuracy"])
    return model 
classifier_name = "cnn"
"""


#simple dense layer model
def MNIST_classifier(ishape = (28,28,1),k = 10, lr = 1e-4):
    model_input = tf.keras.layers.Input(shape = ishape)
    flatten = tf.keras.layers.Flatten()(model_input)
    out = tf.keras.layers.Dense(k,activation = 'softmax')(flatten)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy', metrics = ["accuracy"])
    return model 
classifier_name="dense"    





m = MNIST_classifier()
print(m.summary())
weights = m.get_weights().copy()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
model_name = "mnist_classifier"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss',\
                                             verbose=0,save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 0)

In [5]:
# list to fill output
out_sizes=[]
out_accuracies=[]
out_size_increases=[]

#Experiment parameters:
target_size = 10000
sample_counts = [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]

epoch_count = 30
####-------------------

# iterates through sample sizes and trains classifier for each of them, 
# increasing the dataset size until the target size and
# resetting weights
for size in sample_counts:

    size_increase = target_size-size


    #load data
    X_dev = np.load("data/base_data/MNIST_X_dev_{}_data.npy".format(int(size)))
    y_dev = np.load("data/base_data/MNIST_y_dev_{}_data.npy".format(int(size)))

    #load generator
    gen_model = tf.keras.models.load_model(f"gan_models/trained_gen_{size}.h5")


    #create noise and labels
    noise = tf.random.normal(shape=(int(size_increase), 128))

    noise_pre_labels = np.random.choice(y_dev,size=noise.shape[0])
    noise_labels = tf.keras.utils.to_categorical(noise_pre_labels,10)

    # exception for no increase
    if size_increase>0:
      fake_images = gen_model.predict([noise,noise_labels])

    else:
      fake_images=np.array([])

    # reshape fake images and increase dataset
    fake_images = tf.reshape(fake_images,(-1,28,28))

    new_X_dev = np.array(tf.concat([X_dev,fake_images],axis=0))
    new_y_dev = np.array(tf.concat([y_dev,noise_pre_labels],axis=0))


    #shuffle new dataset
    new_indexes = np.arange(new_X_dev.shape[0],dtype=int)
    np.random.shuffle(new_indexes)
    new_X_dev = new_X_dev[new_indexes]
    new_y_dev = new_y_dev[new_indexes]

    X_test = np.load("data/MNIST_X_test.npy")
    y_test = np.load("data/MNIST_y_test.npy")

    #divide train/split
    n_split = int(0.75*new_X_dev.shape[0])
    print(n_split)
    X_train = new_X_dev[:n_split]
    X_val = new_X_dev[n_split:]
    y_train = new_y_dev[:n_split]
    y_val = new_y_dev[n_split:]

    #one hot encoding
    y_train_oh = tf.keras.utils.to_categorical(y_train)
    y_val_oh = tf.keras.utils.to_categorical(y_val)
    y_test_oh = tf.keras.utils.to_categorical(y_test)

 
    #define model and reset weights
    m = MNIST_classifier()
    m.set_weights(weights)

    # train
    m.fit(X_train,y_train_oh,batch_size = 32, epochs = 50, \
          verbose = 0, callbacks= [early_stop, monitor, lr_schedule],validation_data=(X_val,y_val_oh))

    # evaluate and append results
    metrics = m.evaluate(X_test,y_test_oh)
    print(f"--------------- {size} --------------------")
    print("Categorical cross-entropy:", metrics[0])
    print("Accuracy:", metrics[1])

    out_sizes.append(size)
    out_accuracies.append(metrics[1])
    out_size_increases.append(size_increase)

(9000, 128)
(9000, 10)
7500
313/313 [==============================] - 0s 988us/step - loss: 16.6319 - accuracy: 0.7389
--------------- 1000 --------------------
Categorical cross-entropy: 16.631853103637695
Accuracy: 0.7389000058174133
(8000, 128)
(8000, 10)
7500
313/313 [==============================] - 0s 910us/step - loss: 12.9059 - accuracy: 0.7772
--------------- 2000 --------------------
Categorical cross-entropy: 12.90591812133789
Accuracy: 0.7771999835968018
(7000, 128)
(7000, 10)
7500
313/313 [==============================] - 0s 941us/step - loss: 11.3544 - accuracy: 0.7935
--------------- 3000 --------------------
Categorical cross-entropy: 11.354387283325195
Accuracy: 0.7935000061988831
(6000, 128)
(6000, 10)
7500
313/313 [==============================] - 0s 898us/step - loss: 10.3282 - accuracy: 0.8073
--------------- 4000 --------------------
Categorical cross-entropy: 10.328168869018555
Accuracy: 0.8072999715805054
(5000, 128)
(5000, 10)
7500
313/313 [================

In [6]:
# create new df to add to file
extra_df = pd.DataFrame({
        "original_size":out_sizes,
        "size_increase":out_size_increases,
        "total_size":target_size,
        "accuracy":out_accuracies,
        "training_epochs":epoch_count,
        "model":classifier_name
    })

In [7]:
# append to file
save_entry("10000_fixed.csv",extra_df)